In [54]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F
from datetime import datetime, timedelta

ImportError: cannot import name 'col' from 'pyspark.sql' (/home/skipper/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/sql/__init__.py)

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [9]:
!wc -l data/fhv/csv/fhvhv_tripdata_2021-06.csv

14961893 data/fhv/csv/fhvhv_tripdata_2021-06.csv


In [10]:
df = spark.read \
    .option("header", "true") \
    .csv('data/fhv/csv/fhvhv_tripdata_2021-06.csv')

In [11]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [10]:
!head -n 1001 fhvhv_tripdata_2021-06.csv > head.csv

In [12]:
df_pandas = pd.read_csv('head.csv')
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [12]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [13]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/fhv/csv/fhvhv_tripdata_2021-06.csv')

In [14]:
df = df.repartition(12)

In [17]:
df.write.parquet('data/fhv/pq/', mode='overwrite')

In [18]:
df = spark.read.parquet('data/fhv/pq/')

In [19]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [20]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02617|2021-06-04 16:50:34|2021-06-04 17:01:18|         118|         109|      N|                B02617|
|              B02875|2021-06-02 22:28:45|2021-06-02 22:37:28|         163|          79|      N|                B02875|
|              B02871|2021-06-03 11:47:48|2021-06-03 11:52:23|         231|          13|      N|                B02871|
|              B02888|2021-06-03 08:45:25|2021-06-03 09:00:12|           9|          92|      N|                B02888|
|              B02510|2021-06-05 09:50:43|2021-06-05 10:06:53|          14|         133|      N|                  null|
|              B02764|2021-06-03 22:55:5

In [22]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [24]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [28]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b3b| 2021-06-02|  2021-06-02|         163|          79|
|  e/b48| 2021-06-03|  2021-06-03|           9|          92|
|  e/acc| 2021-06-03|  2021-06-03|         152|          74|
|  e/b38| 2021-06-01|  2021-06-01|         162|         161|
|  s/b13| 2021-06-02|  2021-06-02|          49|         182|
|  e/b35| 2021-06-01|  2021-06-01|         151|          43|
|  e/9ce| 2021-06-03|  2021-06-03|         181|          89|
|  e/b33| 2021-06-04|  2021-06-04|         162|         263|
|  a/a7a| 2021-06-04|  2021-06-04|         125|         164|
|  e/9ce| 2021-06-03|  2021-06-03|         171|          16|
|  e/a39| 2021-06-04|  2021-06-04|         107|         170|
|  a/a7a| 2021-06-01|  2021-06-01|         238|         113|
|  e/b38| 2021-06-03|  2021-06-03|         218|         122|
|  e/9ce| 2021-06-01|  2

In [21]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.Affiliated_base_number == 'B02835') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-06-02 01:21:31|2021-06-02 01:50:23|          49|         182|
|2021-06-03 09:39:41|2021-06-03 09:51:25|         100|         186|
|2021-06-01 21:32:54|2021-06-01 21:42:39|         236|          41|
|2021-06-02 09:01:22|2021-06-02 09:09:12|          48|          90|
|2021-06-03 22:51:28|2021-06-03 23:08:37|         158|         230|
|2021-06-02 15:41:38|2021-06-02 16:28:54|         107|          75|
|2021-06-03 14:37:24|2021-06-03 14:45:20|         260|          82|
|2021-06-02 16:03:52|2021-06-02 17:11:54|         161|         123|
|2021-06-04 09:01:49|2021-06-04 09:15:23|          91|          85|
|2021-06-01 08:14:20|2021-06-01 09:04:23|          67|         261|
|2021-06-02 00:02:28|2021-06-02 00:16:45|         161|           7|
|2021-06-04 17:24:08|2021-06-04 18:01:17|       

In [22]:
df.filter(F.to_date(df.pickup_datetime) == '2021-06-15') \
    .count()

452470

In [ ]:
def time_difference(start, finish):
    t1, t2 = datetime.strptime(start, '%Y-%m-%d %H:%M:%S'), datetime.strptime(finish, '%Y-%m-%d %H:%M:%S')
    return (t2 - t1) / 60

In [38]:
df1 = df.withColumn("date_diff_max", (F.col("dropoff_datetime").cast("long") - F.col("pickup_datetime").cast("long"))/3600)
df1.sort(df1.date_diff_max.desc()).show(truncate=False)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+------------------+
|dispatching_base_num|pickup_datetime    |dropoff_datetime   |PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|date_diff_max     |
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+------------------+
|B02872              |2021-06-25 13:55:41|2021-06-28 08:48:25|98          |265         |N      |B02872                |66.8788888888889  |
|B02765              |2021-06-22 12:09:45|2021-06-23 13:42:44|188         |198         |N      |B02765                |25.549722222222222|
|B02879              |2021-06-27 10:32:29|2021-06-28 06:31:20|78          |169         |N      |B02879                |19.980833333333333|
|B02800              |2021-06-26 22:37:11|2021-06-27 16:49:01|263         |36          |N      |null                  |18.197222222222223|
|B02682              |2021-

In [39]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [40]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [43]:
df2 = df.join(df_zones,df.PULocationID ==  df_zones.LocationID, "inner")

In [55]:
df2.groupBy("Zone") \
  .count() \
  .sort(F.col('count').desc()) \
  .show()

+--------------------+------+
|                Zone| count|
+--------------------+------+
| Crown Heights North|231279|
|        East Village|221244|
|         JFK Airport|188867|
|      Bushwick South|187929|
|       East New York|186780|
|TriBeCa/Civic Center|164344|
|   LaGuardia Airport|161596|
|            Union Sq|158937|
|        West Village|154698|
|             Astoria|152493|
|     Lower East Side|151020|
|        East Chelsea|147673|
|Central Harlem North|146402|
|Williamsburg (Nor...|143683|
|          Park Slope|143594|
|  Stuyvesant Heights|141427|
|        Clinton East|139611|
|West Chelsea/Huds...|139431|
|             Bedford|138428|
|         Murray Hill|137879|
+--------------------+------+
only showing top 20 rows

